In [ ]:
# pip install dlib
# pip install cmake
# pip install opencv-python face_recognition
# pip install face-recognition
# pip install opencv-contrib-python
# pip install --upgrade pip
# pip install tensorflow
# pip install tensorflow keras opencv-python-headless

In [ ]:
# pip install --upgrade opencv-python opencv-python-headless

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [ ]:
# Load model
# model = load_model('fruits-cnn-ad.h5')
model = load_model('fruits-cnn-ad_AllFruits.h5')

In [ ]:
# Print the model summary to check the input shape
model.summary()

In [ ]:
# Mapeo de índices a etiquetas de clases
class_labels = {
    0: 'Apple Healthy', 1: 'Apple Rotten', 2: 'Banana Healthy', 3: 'Banana Rotten',
    4: 'Bellpepper Healthy', 5: 'Bellpepper Rotten', 6: 'Carrot Healthy', 7: 'Carrot Rotten',
    8: 'Cucumber Healthy', 9: 'Cucumber Rotten', 10: 'Grape Healthy', 11: 'Grape Rotten',
    12: 'Guava Healthy', 13: 'Guava Rotten', 14: 'Jujube Healthy', 15: 'Jujube Rotten',
    16: 'Mango Healthy', 17: 'Mango Rotten', 18: 'Orange Healthy', 19: 'Orange Rotten',
    20: 'Pomegranate Healthy', 21: 'Pomegranate Rotten', 22: 'Potato Healthy', 23: 'Potato Rotten',
    24: 'Strawberry Healthy', 25: 'Strawberry Rotten', 26: 'Tomato Healthy', 27: 'Tomato Rotten'
}

In [ ]:
# DataFrame para almacenar los resultados
results_df = pd.DataFrame(columns=['file_name'])

In [ ]:
# Initialize the camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Cannot open the camera")
    exit()

In [ ]:
def preprocess_image(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized_frame = cv2.resize(gray_frame, (98, 98))  # Ensure this matches the input size of your model
    image_array = img_to_array(resized_frame)
    image_array = np.expand_dims(image_array, axis=-1)  # Shape (98, 98, 1)
    image_array = np.expand_dims(image_array, axis=0)  # Shape (1, 98, 98, 1)
    image_array = image_array / 255.0  # Normalize the image
    return image_array

In [ ]:
def predict(image_array):
    preds = model.predict(image_array)
    label = class_labels[np.argmax(preds)]
    prob = np.max(preds)
    return label, prob

In [ ]:
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot receive frame (stream end?). Exiting ...")
        break
    frame = cv2.flip(frame, 1)

    # Preprocess the frame
    image_array = preprocess_image(frame)

    # Make predictions
    try:
        label, prob = predict(image_array)
        label_text = f"{label}: {prob:.2f}"
        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # Green text

        # Print or store the file name
        print("Prediction:", label_text)
        new_row = pd.DataFrame({'file_name': [label_text]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    except Exception as e:
        print(f"Error making prediction: {e}")

    # Show the image
    cv2.imshow('Fruit Detection', frame)

    # Save the frame with prediction
    # cv2.imwrite(os.path.join(save_dir, f'frame_{frame_count}.jpg'), frame)
    # frame_count += 1

    # End capture with 'ESC' key
    if cv2.waitKey(1) & 0xFF == 27:
        break

In [ ]:
# Release resources
cap.release()
cv2.destroyAllWindows()

# Display the results DataFrame
results_df